In [7]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "#1f1e1e"}

import io
import os
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import Markdown, display, Image
from PIL import Image as PILImage

# LOCAL GIT REPOSITORY LOCATION FOR https://github.com/CSSEGISandData/COVID-19
directory = '/Users/davidmorton/Documents/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/'

!git -C $local_covid_19_data pull

states_to_drop = ['Puerto Rico', 'American Samoa', 'Guam', 'District of Columbia', 'Northern Mariana Islands', 'Virgin Islands']

state_names = pd.read_csv('states.csv').set_index('State').to_dict()['Abbreviation']

Already up to date.


In [22]:
def load_csv(directory, f):
    result = pd.read_csv(directory + f)
    result['date'] = datetime.strptime(f, '%m-%d-%Y.csv')
    return result

def get_daily_report_data():
    return pd.concat([load_csv(directory, f) for f in os.listdir(directory) if f.endswith(".csv")])

def get_heatmap_data():
    return (
        get_daily_report_data()         # Get latest from the directory. 
        .copy()                         # Copy from the original to save time loading the data.
        .rename(columns={               # Rename the columns to shorter names
            'Province_State':'state', 
            'Incident_Rate':'rate'
        }) 
        [['state','date','rate']]       # Drop unnecessary columns
        .pivot(                         # Create a pivot table from the data.
            index='date', 
            columns='state', 
            values='rate'
        ) 
        .drop([                         # Keep the 50 states only.
            'Puerto Rico', 
            'American Samoa', 
            'Guam', 
            'District of Columbia', 
            'Northern Mariana Islands', 
            'Virgin Islands', 
            'Diamond Princess', 
            'Grand Princess', 
            'Recovered'
        ], axis=1)   
        .diff()                         # Get the date-to-date difference.
        .clip(0, 30)                    # Clip the data at 30 percent. Some data points are too high. 
        .replace(0, np.nan)             # Get rid of pure 0 values. We'll interpolate these.
        .interpolate()                  # Interpolate the newly NAN values.
        .rolling(window=7).mean()       # 7-Day Rolling Average for smoothness.
        .dropna()                       # Drop the first few rows which are a result of the 7-Day rolling average
        .rename(state_names, axis=1)    # Rename the state names to abbreviations
    )

def make_usa_choropleth(series, locations, title, colorbartitle, colorspectrum, zmax):
    

    fig = go.Figure(
        data=go.Choropleth(
            locations=locations,
            z = series,
            locationmode = 'USA-states',
            colorscale = colorspectrum,
            colorbar_title = colorbartitle, 
            zmax=zmax,
            zmin=0
        )
    )

    fig.update_layout(
        title_text = title,
        geo_scope='usa', # limite map scope to USA
    )

    return fig;

def make_animation(totalduration, pausetime, timeinterpolation, theme):
    bgcolor="black"
    data = get_heatmap_data()
    data = data.rename(state_names, axis=1)
    
    vals = data.values.flatten().flatten()
    vals.sort()
    bins = pd.qcut(vals, q=8, retbins=True)[1]
    colorstops = (bins - bins.min()) / (bins.max() - bins.min())
    colorscale = list(zip(colorstops, getattr(px.colors.sequential, theme)))
    zmax = vals.max()

    data = data.resample(rule=timeinterpolation).asfreq().interpolate()
    
    adjduration = totalduration-pausetime;
    perslide = round((adjduration*1000)/len(data));
    repeatlast = round((pausetime*1000)/perslide)
    print('Frames will be shown for {0} milliseconds'.format(perslide))
    
    images = []
    for i in range(len(data)):
        row = data.iloc[i]
        fig = make_usa_choropleth(row, row.index, 'COVID-19 Confirmed Cases Per 100,000 People - ' + row.name.strftime('%B %d') + ' - {:.0f} Avg'.format(row.mean()), '', colorscale, zmax)
        fig.update_layout(
            plot_bgcolor=bgcolor,
            paper_bgcolor=bgcolor,
            font=dict(color="white"),
            geo = dict(
                lakecolor=bgcolor,
                showlakes=True,
                bgcolor=bgcolor
            )
        )
        img_bytes = fig.to_image(format='png')
        image = PILImage.open(io.BytesIO(img_bytes))

        images.append(image)
        if (i == (len(data) - 1)):
            for j in range(1,repeatlast):
                images.append(image)
        
    url = 'covid19-map-case-incidence-rate.gif'
    images[0].save(url,
                   save_all=True, append_images=images[1:], optimize=True, duration=perslide, loop=0)
    return url + "?c=" + str(random.randint(0,2e9))


In [23]:
url = make_animation(30, 5, '6H', "Inferno")
Image(url=url)

Frames will be shown for 83 milliseconds
